In [6]:
!pip install findspark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=8c022009199d60275218884da1ebffaa267395fc108f8489e8ffebbd6a31ef13
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [7]:
# Import findspark and initialize.
import findspark
findspark.init()

In [8]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [13]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile("https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv")
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True)

In [14]:
# 2. Create a temporary view of the DataFrame.

df.createOrReplaceTempView("home_sales")


In [16]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""
    SELECT YEAR(date) as year, ROUND(AVG(price), 2) as avg_price
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY YEAR(date)
    ORDER BY year
""").show()



+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [18]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
result = spark.sql("""
    SELECT YEAR(date_built) as year_built, ROUND(AVG(price), 2) as avg_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY year_built
""")
result.show()



+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2015| 288770.3|
|      2013|295962.27|
|      2014|290852.27|
|      2012|293683.19|
|      2016|290555.07|
|      2010|292859.62|
|      2011|291117.47|
|      2017|292676.79|
+----------+---------+



In [22]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
result = spark.sql("""
SELECT YEAR(date_built) as year_built, ROUND(AVG(price), 2) as avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY YEAR(date_built)
ORDER BY year_built
""")
result.show()





+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [25]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()
result = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) as avg_price
    FROM home_sales
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view
""")
result.show()
end_time = time.time()

print("Query Runtime: {} seconds".format(end_time - start_time))

+----+---------+
|view|avg_price|
+----+---------+
|   0|403848.51|
|   1|401044.25|
|  10|401868.43|
| 100|1026669.5|
|  11|399548.12|
|  12|401501.32|
|  13|398917.98|
|  14|398570.03|
|  15| 404673.3|
|  16|399586.53|
|  17|398474.49|
|  18|399332.91|
|  19|398953.17|
|   2|397389.25|
|  20|399522.81|
|  21|399758.88|
|  22|402022.68|
|  23|403411.92|
|  24|400284.92|
|  25|401298.69|
+----+---------+
only showing top 20 rows

Query Runtime: 1.1359097957611084 seconds


In [26]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

DataFrame[]

In [27]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [29]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()
result = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) as avg_price
    FROM home_sales
    WHERE price >= 350000
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY view
""")
result.show()

end_time = time.time()
cached_runtime = end_time - start_time
print("Query Runtime on Cached Data: {} seconds".format(cached_runtime))


+----+---------+
|view|avg_price|
+----+---------+
|   0|403848.51|
|   1|401044.25|
|  10|401868.43|
| 100|1026669.5|
|  11|399548.12|
|  12|401501.32|
|  13|398917.98|
|  14|398570.03|
|  15| 404673.3|
|  16|399586.53|
|  17|398474.49|
|  18|399332.91|
|  19|398953.17|
|   2|397389.25|
|  20|399522.81|
|  21|399758.88|
|  22|402022.68|
|  23|403411.92|
|  24|400284.92|
|  25|401298.69|
+----+---------+
only showing top 20 rows

Query Runtime on Cached Data: 0.3873260021209717 seconds


In [30]:
import time

# Measure uncached runtime
spark.catalog.uncacheTable("home_sales")  # Ensure the table is not cached
start_time = time.time()
# Run the query
# ...
uncached_runtime = time.time() - start_time

# Measure cached runtime
spark.catalog.cacheTable("home_sales")  # Ensure the table is cached
start_time = time.time()
# Run the same query
# ...
cached_runtime = time.time() - start_time

# Compare the two
print("Uncached Query Runtime: {} seconds".format(uncached_runtime))
print("Cached Query Runtime: {} seconds".format(cached_runtime))
print("Difference: {} seconds".format(uncached_runtime - cached_runtime))


Uncached Query Runtime: 6.031990051269531e-05 seconds
Cached Query Runtime: 6.532669067382812e-05 seconds
Difference: -5.0067901611328125e-06 seconds


In [31]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").parquet("path_to_save")

In [32]:
# 11. Read the formatted parquet data.
parquet_df = spark.read.parquet("path_to_save")

In [33]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("parquet_home_sales")

In [36]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

import time
start_time = time.time()

result = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) as avg_price
    FROM parquet_home_sales
    WHERE price >= 350000
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY view
""")
result.show()

parquet_runtime = time.time() - start_time
print("Query Runtime on Parquet Data: {} seconds".format(parquet_runtime))

# Assuming 'cached_runtime' is a previously measured runtime for the same query on cached data
print("Cached Query Runtime: {} seconds".format(cached_runtime))
print("Difference: {} seconds".format(parquet_runtime - cached_runtime))

# Optionally, stop the SparkSession if it's no longer needed
spark.stop()


+----+---------+
|view|avg_price|
+----+---------+
|   0|403848.51|
|   1|401044.25|
|  10|401868.43|
| 100|1026669.5|
|  11|399548.12|
|  12|401501.32|
|  13|398917.98|
|  14|398570.03|
|  15| 404673.3|
|  16|399586.53|
|  17|398474.49|
|  18|399332.91|
|  19|398953.17|
|   2|397389.25|
|  20|399522.81|
|  21|399758.88|
|  22|402022.68|
|  23|403411.92|
|  24|400284.92|
|  25|401298.69|
+----+---------+
only showing top 20 rows

Query Runtime on Parquet Data: 1.0125370025634766 seconds
Cached Query Runtime: 6.532669067382812e-05 seconds
Difference: 1.0124716758728027 seconds


In [37]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [38]:
# 15. Check if the home_sales is no longer cached

is_cached = spark.catalog.isCached("home_sales")
print(is_cached)


False
